# Encoding functions

This notebook includes the functions for encoding a dictionary into HD vectors and an associative memory. In order to develop the program I'm using a toy set of object descriptions from McRae's dataset.

In [76]:
%run HDComputing_basics.ipynb

#init()

Dict_defs = {'bowl':         [['is','kitchen_utensil'], ['shape','round'],['used_for','eating'],['material','plastic']],
             'cup' :         [['is','kitchen_utensil'], ['shape','cylinder'],['used_for','drinking'],['has','handle'],['material','ceramic']],
             'fork':         [['is','kitchen_utensil'], ['used_for','eating'],['has','prongs'],['material','metal']],
             'knife':        [['is','kitchen_utensil'], ['used_for','cutting'],['has','handle'],['material','metal']],
             'plate':        [['is','kitchen_utensil'], ['used_for','eating'],['shape','round'],['material','ceramic']],
             'pot':          [['is','kitchen_utensil'], ['used_for','cooking'],['has','handle'],['shape','round']],
             'spoon':        [['is','kitchen_utensil'], ['used_for','eating'],['material','metal'],['has','round_end']],
            
             'apple':        [['color','red'],['is','fruit'],['color','green'], ['taste','sweet'], ['origin','trees'],['consistency','crunchy']]
            }   

feature_vectors = []  # List of feature vectors such as 'is', 'used_for', 'shape', etc...

## Auxiliar functions

In [77]:
def flat_list (L):
    "Recursive function that flats a list of lists (at any level)"
    if L == []:
        return L
    if type(L[0]) is list:
        return flat_list(L[0]) + flat_list(L[1:])
    return L[:1] + flat_list(L[1:])

## Memory functions

In [78]:
def CreateSemanticPointer (PairList):
    "Turns list as [[feat1,feat_val],[feat2,feat_val],[feat3,feat_val]] into vector feat1*feat_val + feat2*feat_val ..."
    vecs = []
    for pair in PairList:
        vecs.append(Dict[pair[0]] * Dict[pair[1]])
    return ADD(vecs)
            
def SaveConcepts(Dic):
    """Given a definitions dictionary it stores in memory the entire set of concepts in the dictionary (including feature vectors)"""
    keys = Dic.keys()
    vals = Dic.values()
    all_concepts = set(flat_list(vals) + keys)
    # Process for storing list of concepts in memory
    for concept in all_concepts:
        HDvector(N,concept) #This creates an object and store it in memory
        
def SaveDefinitions(Dic):
    """Given the definitions dictionary, and having all its concepts previously stored in memory, this functions
       creates a definition vector (semantic pointer) using HD operations and assign it as a pointer to an 
       object vector (ID vector)."""
    global feature_vectors
    vecs = []  #List for creating the semantic pointer
    for key, value in Dic.iteritems():
        for pair in value:  # It goes through each pair [feature_vector, feature_value]
            vecs.append(Dict[pair[0]] * Dict[pair[1]])  # Binding operation
            feature_vectors.append(pair[0])
        Dict[key].setPointer(ADD(vecs)) # Asign the semantic pointer vector to ID vector by using adding all vectors from vecs
        vecs = []   #Clean it up for next iteration
    feature_vectors = list(set(feature_vectors))  #Delete repeated elements

## Testing encoding

In [79]:
def EncodingTests():
    init()
    SaveConcepts(Dict_defs)
    SaveDefinitions(Dict_defs)
    
    # Comparing a definition vs a single feature vector  -> orthogonal...
    d1 = Dict['bowl'].getPointer().dist( Dict['shape'] )  
    print "Distance from semantic pointer of 'bowl' to 'shape':", d1
    assert d1 > 0.4 * N
    
    # Comparing an unbinded SP vector vs an specific feature value -> same vector
    #d2 = Dict['round'].dist( Dict['bowl'].getPointer() * Dict['shape'] )
    d2 = Dict['round'].dist( Dict['bowl'] ** Dict['shape'])  # Equivalent to previous line... 
    print "Distance from semantic pointer 'bowl'*'shape' to 'round':", d2
    print "Assigned label: ", HDvector.getLabel( Dict['bowl'] ** Dict['shape'] )
    assert d2 < 0.4 * N
    
    d22 = Dict['red'].dist( Dict['apple'] ** Dict['color']) 
    print "Distance from semantic pointer 'apple'*'color' to 'red':", d22
    print "Assigned label: ", HDvector.getLabel( Dict['apple'] ** Dict['color'] )
    assert d22 < 0.4 * N
    
    d23 = Dict['green'].dist( Dict['apple'] ** Dict['color']) 
    print "Distance from semantic pointer 'apple'*'color' to 'green':", d23
    print "Assigned label: ", HDvector.getLabel( Dict['apple'] ** Dict['color'] )
    assert d23 < 0.4 * N
    
    
    # Comparing two semantic pointer vectors:
    d3 = HDvector.dist( Dict['knife'].getPointer(), Dict['spoon'].getPointer() )
    print "Distance from knife to spoon (definitions):", d3
    d4 = HDvector.dist( Dict['knife'].getPointer(), Dict['pot'].getPointer() )
    print "Distance from knife to pot (definitions):", d4
    
    # Asking for closest concept
    vec = CreateSemanticPointer( [['shape','round'], ['material','metal'],['used_for','eating']] )
    print Dict['fork'].getPointer().dist(vec)
    
    #FALTA IMPLEMENTAR UNA FUNCIÓN LLAMADA ALGO ASÍ COMO get ID vector... no debe dar uno sino varios
    # TAMBIÉN EL DE GET LABEL debe dar mejor una lista de los más cercanos (que pasan cierto umbral) 
    # puede ser solo la lista o una lista de listas [concepto, distancia]
    
    # ES IMPORTANTE QUE REGRESE LAS VARIAS OPCIONES (DE COLOR o Material por ejemplo) pues eso dará pie a ampliar la busqueda
    
    # Hacer pruebas con el límite de 'compresión', es decir, probar con definiciones largas y ver si aun se recupera bien
    # la información... puedo incluso hacer tablas de eso...
    
    # Tampoco estaría mal hacer un programa que haga una matriz de distancias de todos los semantic pointers contra los demás...
    
    # Sería interesante probar la codificación de un grafo según gayler:  G = A * P(B) + A * P(C) + B * P(D)
    # (si el vector es permutado es hijo, el no permutado es padre)
    # ¿qué ventajas hay?, se ve que es más sencillo pero quien sabe que desventajas tenga....
    
    #print vec.getLabel()
EncodingTests()

Distance from semantic pointer of 'bowl' to 'shape': 5045
Distance from semantic pointer 'bowl'*'shape' to 'round': 3108
Assigned label:  round
Distance from semantic pointer 'apple'*'color' to 'red': 3398
Assigned label:  red
Distance from semantic pointer 'apple'*'color' to 'green': 3431
Assigned label:  red
Distance from knife to spoon (definitions): 3574
Distance from knife to pot (definitions): 3563
3121
